In [1]:
import pandas as pd
pd.options.display.max_columns = None

from scripts.data_processing import (
    load_uci, load_tidepool_dummy, 
    load_so_pump, load_so_cgm
)

from scripts.database import (
    get_db_conn, initial_schema_creation
)    

In [ ]:
df = load_uci()
print(df.shape)
df.head()

In [ ]:
df = load_tidepool_dummy()
print(df.shape)
df.head()

In [ ]:
df = load_so_pump_raw()
print(df.shape)
df.head()

In [ ]:
df = load_so_pump()
print(df.shape)
df.head()

In [2]:
df = load_so_cgm()
print(df.shape)
df.head()

(92055, 3)


,timestamp,measurement,below_threshold
9,2018-04-26T00:02:36,80,False
10,2018-04-26T00:07:37,83,False
11,2018-04-26T00:12:36,86,False
12,2018-04-26T00:17:37,89,False
13,2018-04-26T00:22:37,93,False


In [ ]:
# these duplicates are problematic
(df.loc[df['Timestamp (YYYY-MM-DDThh:mm:ss)'].duplicated(keep=False)]
   .sort_values('Timestamp (YYYY-MM-DDThh:mm:ss)')
   .head())